In [20]:
import pandas as pd

In [26]:
df = pd.read_csv("resultados_simulacion.csv")
df.head()

,ID de simulación,Numero de cliente,Hora de llegada,Tiempo entre llegada,Tiempo de llegada,Tiempo en Cola,Hora de inicio de servicio,Tiempo de servicio,Hora de fin de servicio,Tiempo en el sistema,Longitud del sistema,Longitud de la Cola
0,1,Cliente-1,0:21:21,0.000000,1281.464447,0.0,0:21:21,450.621406,0:28:52,450.621406,0,0
1,1,Cliente-2,0:29:18,476.595646,1758.060093,0.0,0:29:18,388.102274,0:35:46,388.102274,0,0
2,1,Cliente-3,0:37:12,474.436131,2232.496223,0.0,0:37:12,225.750816,0:40:58,225.750816,0,0
3,1,Cliente-4,0:44:05,412.637698,2645.133922,0.0,0:44:05,31.111719,0:44:36,31.111719,0,0
4,1,Cliente-5,1:03:46,1181.317716,3826.451638,0.0,1:03:46,6.899727,1:03:53,6.899727,0,0


In [22]:
# Agrupar por ID de simulación
agrupado_por_simulacion = df.groupby('ID de simulación')

# Calcular métricas por simulación
resultados = agrupado_por_simulacion.agg(
    longitud_esperada_sistema=('Longitud del sistema', 'mean'),
    longitud_promedio_cola=('Longitud de la Cola', 'mean'),
    tiempo_total_espera=('Tiempo de servicio', 'sum'),
    tiempo_promedio_sistema=('Tiempo en el sistema', 'mean'),
    tiempo_total_cola=('Tiempo en Cola', 'sum'),
    promedio_tiempo_cola=('Tiempo en Cola', 'mean'),
    maxima_longitud_cola=('Longitud de la Cola', 'max')
).reset_index()

In [23]:
resultados.head()

,ID de simulación,longitud_esperada_sistema,longitud_promedio_cola,tiempo_total_espera,tiempo_promedio_sistema,tiempo_total_cola,promedio_tiempo_cola,maxima_longitud_cola
0,1,1.073171,0.170732,24886.641005,376.661434,5999.596610,73.165812,2
1,2,1.980198,0.772277,33277.284427,543.913138,21657.942548,214.435075,6
2,3,0.835821,0.119403,20214.330916,373.688465,4822.796255,71.982034,3
3,4,0.916667,0.095238,21297.948564,312.616244,4961.815945,59.069237,2
4,5,0.697368,0.039474,18214.475618,265.137570,1935.979688,25.473417,1


## Explicación del código

### Importación de módulos
```python
import simpy
import random
import csv
from datetime import timedelta
from pathlib import Path
```
- **simpy**: Utilizado para la simulación de procesos discretos.
- **random**: Usado para generar tiempos aleatorios de llegada y servicio de los clientes.
- **csv**: Para escribir los resultados de la simulación en un archivo CSV.
- **timedelta**: Para convertir los tiempos de simulación de segundos a formato HH:MM:SS.
- **Path**: Para manejar rutas de archivos de forma robusta.

### Función `formato_hora`
```python
def formato_hora(segundos):
    """Convierte segundos a formato HH:MM:SS."""
    return str(timedelta(seconds=round(segundos)))
```
Esta función toma un valor en segundos y lo convierte al formato estándar de hora `HH:MM:SS`, redondeando el valor para evitar fracciones de segundo.

### Función `cliente`
```python
def cliente(env, nombre, servidores, tasa_servicio, datos_cliente):
    """Proceso que representa un cliente."""
    llegada = env.now
    tiempo_entre_llegada = llegada - datos_cliente['ultima_llegada'] if datos_cliente['ultima_llegada'] is not None else 0
    datos_cliente['ultima_llegada'] = llegada

    longitud_sistema = len(servidores.queue) + (servidores.count if servidores.count > 0 else 0)
    longitud_cola = len(servidores.queue)

    print(f"{nombre} llegó a las {formato_hora(llegada)}")
    
    with servidores.request() as req:
        yield req
        tiempo_en_cola = env.now - llegada
        inicio_servicio = env.now
        
        tiempo_servicio = random.expovariate(tasa_servicio)
        yield env.timeout(tiempo_servicio)
        
        fin_servicio = env.now
        
        datos_cliente['clientes'].append({
            'ID de simulación': datos_cliente['id_simulacion'],
            'Numero de cliente': nombre,
            'Hora de llegada': formato_hora(llegada),
            'Tiempo entre llegada': tiempo_entre_llegada,
            'Tiempo de llegada': llegada,
            'Tiempo en Cola': tiempo_en_cola,
            'Hora de inicio de servicio': formato_hora(inicio_servicio),
            'Tiempo de servicio': tiempo_servicio,
            'Hora de fin de servicio': formato_hora(fin_servicio),
            'Tiempo en el sistema': tiempo_en_cola + tiempo_servicio,
            'Longitud del sistema': longitud_sistema,
            'Longitud de la Cola': longitud_cola
        })
```
Esta función simula el comportamiento de un cliente dentro de un sistema de servidores:
- Un cliente llega al sistema y se registra la hora de llegada.
- Se calcula el tiempo que el cliente ha esperado en la cola.
- El cliente se atiende en el servidor, donde el tiempo de servicio es aleatorio (distribución exponencial).
- Se registran diversos datos sobre el cliente, como la hora de llegada, el tiempo en la cola, el tiempo de servicio, y más.

### Función `generar_clientes`
```python
def generar_clientes(env, tasa_llegada, tasa_servicio, servidores, datos_cliente):
    """Genera clientes con tiempos de llegada exponenciales."""
    i = 0
    while True:
        yield env.timeout(random.expovariate(tasa_llegada))
        i += 1
        env.process(cliente(env, f"Cliente-{i}", servidores, tasa_servicio, datos_cliente))
```
Esta función genera clientes en intervalos de tiempo aleatorios (distribución exponencial) y los agrega al proceso de simulación. El número de clientes está determinado por la tasa de llegada.

### Función `simulacion`
```python
def simulacion(tasa_llegada, tasa_servicio, n_servidores, tiempo_simulacion, id_simulacion):
    """Ejecuta la simulación y retorna los datos recolectados."""
    env = simpy.Environment()
    servidores = simpy.Resource(env, capacity=n_servidores)
    datos_cliente = {'ultima_llegada': None, 'clientes': [], 'id_simulacion': id_simulacion}
    
    env.process(generar_clientes(env, tasa_llegada, tasa_servicio, servidores, datos_cliente))
    env.run(until=tiempo_simulacion)

    return datos_cliente['clientes']
```
Aquí se configura y ejecuta la simulación:
- Se inicializa el entorno de SimPy.
- Se crea el recurso de servidores con la capacidad especificada (`n_servidores`).
- Se genera un flujo continuo de clientes con la función `generar_clientes`.
- Se corre la simulación hasta el tiempo especificado (`tiempo_simulacion`).
- Se devuelven los datos recolectados sobre los clientes.

### Función `guardar_resultados_csv`
```python
def guardar_resultados_csv(datos_clientes, archivo_csv="resultados_simulacion.csv"):
    """Guarda los resultados de la simulación en un archivo CSV en la misma carpeta del script."""
    columnas = [...]
    ruta_script = Path(__file__).parent
    ruta_csv = ruta_script / archivo_csv
    
    escribir_encabezado = not ruta_csv.exists()

    with ruta_csv.open(mode='a', newline='', encoding='utf-8') as archivo:
        escritor = csv.DictWriter(archivo, fieldnames=columnas)
        if escribir_encabezado:
            escritor.writeheader()
        escritor.writerows(datos_clientes)
```
Esta función guarda los resultados de la simulación en un archivo CSV. Si el archivo ya existe, simplemente agrega nuevos registros; si no, escribe un encabezado con los nombres de las columnas.

### Función `convertir_parametros`
```python
def convertir_parametros(hora_inicio, hora_fin, clientes_por_hora, capacidad_servicio_por_hora, n_servidores):
    """Convierte los parámetros de la simulación al formato requerido en segundos."""
    ...
    return {...}
```
Convierte los parámetros legibles (como horas) a valores en segundos y ajusta las tasas de llegada y servicio a valores por segundo.

### Función `obtener_ultimo_id_simulacion`
```python
def obtener_ultimo_id_simulacion(archivo_csv="resultados_simulacion.csv"):
    """Obtiene el último ID de simulación del archivo CSV, si existe."""
    ...
```
Obtiene el último ID de simulación registrado en el archivo CSV, lo que permite que las simulaciones futuras se numeren consecutivamente.

### Función `ejecutar_simulaciones`
```python
def ejecutar_simulaciones(cantidad_simulaciones, parametros, archivo_csv="resultados_simulacion.csv"):
    """Ejecuta múltiples simulaciones y las guarda en un archivo CSV."""
    ...
```
Ejecuta múltiples simulaciones con parámetros dados, numerando las simulaciones de manera secuencial y guardando los resultados en un archivo CSV.

### Ejecución final de simulaciones
```python
parametros_simulacion = convertir_parametros(...)
ejecutar_simulaciones(3, parametros_simulacion)
```
Finalmente, el código convierte los parámetros legibles a un formato adecuado y ejecuta 3 simulaciones, guardando los resultados en un archivo CSV.

### Conclusión
Este código simula un sistema de servidores atendiendo clientes con tiempos de llegada y servicio aleatorios. Los resultados se guardan en un archivo CSV para su análisis posterior.